In [64]:
import datetime
import sklearn
import typing as tp
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

In [65]:
X_type = tp.NewType("X_type", np.ndarray)
X_row_type = tp.NewType("X_row_type", np.ndarray)
Y_type = tp.NewType("Y_type", np.array)
TS_type = tp.NewType("TS_type", pd.Series)
Model_type = tp.TypeVar("Model_type")

In [66]:
def read_timeseries(path_to_df: str = "train.csv") -> TS_type:
    """Функция для чтения данных и получения обучающей и тестовой выборок"""
    df = pd.read_csv(path_to_df)
    df = df[(df['store'] == 1) & (df['item'] == 1)]
    df["date"] = pd.to_datetime(df["date"])
    df = df.set_index("date")
    ts = df["sales"]
    train_ts = ts[:-365]
    test_ts = ts[-365:]
    return train_ts, test_ts

In [67]:
def extract_hybrid_strategy_features(
    timeseries: TS_type,
    model_idx: int,
    window_size: int = 7
) -> X_row_type:
    """
    Функция для получения вектора фичей согласно гибридной схеме. На вход подаётся временной ряд
    до момента T, функция выделяет из него фичи, необходимые модели под номером model_idx для
    прогноза на момент времени T
    
    Args:
        timeseries --- временной ряд до момента времени T (не включительно), pd.Series с датой 
                       в качестве индекса
        model_idx --- индекс модели, то есть номер шага прогноза, 
                      для которого нужно получить признаки, нумерация с нуля
        window_size --- количество последних значений ряда, используемых для прогноза 
                        (без учёта количества прогнозов с предыдущих этапов)

    Returns:
        Одномерный вектор фичей для модели с индексом model_idx (np.array), 
        чтобы сделать прогноз для момента времени T
    """
    feature_window = window_size + model_idx
    return timeseries[-feature_window:].values

In [68]:
def build_datasets(
    timeseries: TS_type,
    extract_features: tp.Callable[..., X_row_type],
    window_size: int,
    model_count: int
) -> tp.List[tp.Tuple[X_type, Y_type]]:
    """
    Функция для получения обучающих датасетов согласно гибридной схеме
    
    Args:
        timeseries --- временной ряд
        extract_features --- функция для генерации вектора фичей
        window_size --- количество последних значений ряда, используемых для прогноза
        model_count --- количество моделей, используемых для получения предскзаний 

    Returns:
        Список из model_count датасетов, i-й датасет используется для обучения i-й модели 
        и представляет собой пару из двумерного массива фичей и одномерного массива таргетов
    """
    datasets = []
    
    for model_idx in range(model_count):
        features = []
        labels = []
        for k in range(window_size + model_idx, len(timeseries)):
            features.append(extract_features(timeseries[:k], model_idx, window_size))
            labels.append(timeseries.iloc[k])
        features = np.array(features)
        labels = np.array(labels)
    
        datasets.append((features, labels))
    
    assert len(datasets) == model_count
    return datasets

In [69]:
def predict(
    timeseries: TS_type,
    models: tp.List[Model_type],
    extract_features: tp.Callable[..., X_row_type] = extract_hybrid_strategy_features
) -> TS_type:
    """
    Функция для получения прогноза len(models) следующих значений временного ряда
    
    Args:
        timeseries --- временной ряд, по которому необходимо сделать прогноз на следующие даты
        models --- список обученных моделей, i-я модель используется для получения i-го прогноза
        extract_features --- функция для генерации вектора фичей. Если вы реализуете свою функцию 
                             извлечения фичей для конечной модели, передавайте этим аргументом.
                             Внутри функции predict функцию extract_features нужно вызывать только
                             с аргументами timeseries и model_idx, остальные должны быть со значениями
                             по умолчанию

    Returns:
        Прогноз len(models) следующих значений временного ряда
    """
    
    
    preds = []
    for model_idx, model in enumerate(models):
        features = extract_features(timeseries, model_idx)
        pred = model.predict(features.reshape(1, -1)) 
        
        new_index = pd.date_range(timeseries.index[-1], periods=2)[1:]
#         timeseries = timeseries.append(pd.Series(pred, index=new_index))
        timeseries = pd.concat([timeseries, pd.Series(pred, index=new_index)], copy=False)
                
    return timeseries[-len(models):]

In [70]:
def train_models(
    train_timeseries: TS_type,
    model_count: int
) -> tp.List[Model_type]:
    """
    Функция для получения обученных моделей
    
    Args:
        train_timeseries --- обучающий временной ряд
        model_count --- количество моделей для обучения согласно гибридной схеме.
                        Прогнозирование должно выполняться на model_count дней вперёд

    Returns:
        Список из len(datasets) обученных моделей
    """
    models = []

    # datasets = build_datasets(train_timeseries, ...)
    # YOUR CODE HERE
    datasets = build_datasets(train_timeseries, extract_hybrid_strategy_features, window_size=7, model_count=model_count)  
    
    for X_train, y_train in datasets:
        random_forest = sklearn.ensemble.RandomForestRegressor(n_estimators=10, random_state=42)
        random_forest.fit(X_train, y_train)
        models.append(random_forest)
    
    assert len(models) == len(datasets)
    return models

In [71]:
def score_models(
    train_ts: TS_type,
    test_ts: TS_type, 
    models: tp.List[Model_type],
    predict: tp.Callable[[TS_type, tp.List[Model_type]], TS_type] = predict
):
    """
    Функция для оценки качества обученных моделей по метрике MSE
    
    Args:
        train_ts --- обучающий временной ряд
        test_ts --- тестовый временной ряд
        models --- список обученных моделей
        predict --- функция для получения прогноза временного ряда

    Returns:
        Усредненное MSE для прогноза моделей по всей тестовой выборке
    """

    inferred_freq = pd.infer_freq(train_ts.index)
    
    predict_len = len(models)
    predictions = []
    targets = []

    for i in range(len(test_ts) - predict_len + 1):
                
        predictions.extend(list(predict(train_ts, models)))
        targets.extend(list(test_ts[i : i + predict_len]))

        next_date = train_ts.index[-1] + pd.tseries.frequencies.to_offset(inferred_freq)
        train_ts.loc[next_date] = test_ts[i : i + 1].values[0]
    
    return sklearn.metrics.mean_squared_error(targets, predictions)

In [72]:
data_train, data_test = read_timeseries()

In [73]:
data_train.head()

date
2013-01-01    13
2013-01-02    11
2013-01-03    14
2013-01-04    13
2013-01-05    10
Name: sales, dtype: int64

In [74]:
data_test.head()

date
2017-01-01    19
2017-01-02    15
2017-01-03    10
2017-01-04    16
2017-01-05    14
Name: sales, dtype: int64

In [75]:
models = train_models(data_train, 4)

In [76]:
models

[RandomForestRegressor(n_estimators=10, random_state=42),
 RandomForestRegressor(n_estimators=10, random_state=42),
 RandomForestRegressor(n_estimators=10, random_state=42),
 RandomForestRegressor(n_estimators=10, random_state=42)]

In [77]:
score_models(data_train, data_test, models, predict)

34.43051795580111